# Azure AI Agents with Model Context Protocol (MCP) Support - Python

このノートブックでは、PythonでAzure AI AgentsをModel Context Protocol (MCP) ツールと共に使用する方法を示します。外部のMCPサーバー（例えばMicrosoft Learn）を活用し、キー不要の認証を使用して機能を強化するインテリジェントエージェントを作成する方法を紹介します。


## 必要なPythonパッケージのインストール

まず、必要なPythonパッケージをインストールします:
- **azure-ai-projects**: Azure AI ProjectsのコアSDK
- **azure-ai-agents**: エージェントの作成と管理用Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredentialを使用したキー不要の認証を提供
- **mcp**: Python向けのModel Context Protocol実装


## キーレス認証のメリット

このノートブックでは、**キーレス認証**のデモを行います。これには以下のような多くの利点があります：
- ✅ **APIキーの管理が不要** - Azureのアイデンティティベース認証を使用
- ✅ **セキュリティの向上** - コードや設定ファイルに秘密情報を保存しない
- ✅ **資格情報の自動ローテーション** - Azureが資格情報のライフサイクル管理を担当
- ✅ **ロールベースのアクセス制御** - Azure RBACを使用して細かい権限設定が可能
- ✅ **マルチ環境対応** - 開発環境と本番環境の両方でシームレスに動作

`DefaultAzureCredential` は、利用可能な最適な資格情報ソースを自動的に選択します：
1. **マネージドID**（Azure上で実行されている場合）
2. **Azure CLI** の資格情報（ローカル開発時）
3. **Visual Studio** の資格情報
4. **環境変数**（設定されている場合）
5. **インタラクティブブラウザ**認証（フォールバックとして）


## キーレス認証のセットアップ

**キーレス認証の前提条件:**

### ローカル開発の場合:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure環境の場合:
- Azureリソースで**システム割り当てマネージドID**を有効化する
- マネージドIDに適切な**RBACロール**を割り当てる:
  - Azure OpenAIアクセス用の`Cognitive Services OpenAI User`
  - Azure AIプロジェクトアクセス用の`AI Developer`

### 環境変数 (オプション):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**APIキーや接続文字列は不要です！** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## 必要なライブラリのインポート

必要なPythonモジュールをインポートします:
- **os, time**: 環境変数や遅延処理のための標準Pythonライブラリ
- **AIProjectClient**: Azure AI Projectsのメインクライアント
- **DefaultAzureCredential**: Azureサービスのキー不要認証
- **MCP関連クラス**: MCPツールの作成と管理、承認処理のため


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCPサーバー設定の構成

環境変数を使用してMCPサーバーの設定を構成し、デフォルト値をフォールバックとして設定します:
- **MCP_SERVER_URL**: MCPサーバーのURL (デフォルトはMicrosoft Learn API)
- **MCP_SERVER_LABEL**: MCPサーバーを識別するためのラベル (デフォルトは "mslearn")

この方法により、異なる環境で柔軟な構成が可能になります。


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI プロジェクトクライアントの作成（キー不要の認証）

**キー不要の認証**を使用して Azure AI プロジェクトクライアントを初期化します:
- **endpoint**: Azure AI Foundry プロジェクトのエンドポイント URL
- **credential**: `DefaultAzureCredential()` を使用して安全なキー不要の認証を実現
- **API キーは不要**: 利用可能な最適な資格情報を自動的に検出して使用

**認証フロー:**
1. マネージド ID を確認（Azure 環境内）
2. Azure CLI 資格情報にフォールバック（ローカル開発用）
3. 必要に応じて他の利用可能な資格情報ソースを使用

このアプローチにより、コード内で API キーや接続文字列を管理する必要がなくなります。


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCPツール定義の作成

Microsoft Learn MCPサーバーに接続するMCPツールを作成します:
- **server_label**: MCPサーバーの識別子
- **server_url**: MCPサーバーのURLエンドポイント
- **allowed_tools**: 使用可能なツールを制限するためのオプションのリスト（空のリストの場合、すべてのツールが許可されます）

このツールを使用することで、エージェントがMicrosoft Learnのドキュメントやリソースにアクセスできるようになります。


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## エージェントの作成と会話の実行（キー不要のワークフロー）

この包括的なセクションでは、**キー不要のエージェントワークフロー**の全体像を示します：

1. **AIエージェントの作成**: GPT-4.1 nanoモデルとMCPツールを使用してエージェントを設定
2. **スレッドの作成**: コミュニケーション用の会話スレッドを確立
3. **メッセージの送信**: Azure OpenAIとOpenAIの違いについてエージェントに質問
4. **ツール承認の処理**: 必要に応じてMCPツールの呼び出しを自動承認
5. **実行の監視**: エージェントの進捗を追跡し、必要なアクションを処理
6. **結果の表示**: 会話内容とツール使用の詳細を表示

**キー不要の特徴:**
- ✅ **ハードコードされた秘密情報なし** - すべての認証はAzure IDで処理
- ✅ **デフォルトで安全** - ロールベースのアクセス制御を使用
- ✅ **簡素化されたデプロイ** - 資格情報管理が不要
- ✅ **監査対応** - すべてのアクセスはAzure IDを通じて追跡

エージェントはMCPツールを使用してMicrosoft Learnリソースにアクセスし、完全なセキュリティとAPIキー管理不要の状態を実現します。


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
